# Pneumonia-Treatment-RAG

In [ ]:
!pip install langchain_openai langchain openai datasets qdrant-client tiktoken

In [ ]:
!pip install -U langchain-community

In [3]:
from langchain.document_loaders import PyPDFLoader

documents = PyPDFLoader('/kaggle/input/pneumoniadocuments/(Public Health in the 21st Century) Micaela L. Surez_ Steffani M. Ortega - Pneumonia_ Symptoms Diagnosis and Treatment _ Symptoms Diagnosis and Treatment-Nova Science Publishers Incorporated (2011.pdf').load()

In [4]:
from langchain.text_splitter import CharacterTextSplitter

chunks = CharacterTextSplitter(chunk_size= 1000, chunk_overlap=50).split_documents(documents)

In [5]:
chunks[0]

Document(metadata={'source': '/kaggle/input/pneumoniadocuments/(Public Health in the 21st Century) Micaela L. Surez_ Steffani M. Ortega - Pneumonia_ Symptoms Diagnosis and Treatment _ Symptoms Diagnosis and Treatment-Nova Science Publishers Incorporated (2011.pdf', 'page': 0}, page_content='Pneumonia: Symptoms, Diagnosis and Treatment : Symptoms, Diagnosis and Treatment, edited by Micaela L. Suárez, and Steffani M. Ortega, Nova\nCopyright © 2011. Nova Science Publishers, Incorporated. All rights reserved.')

In [6]:
chunks[0].metadata

{'source': '/kaggle/input/pneumoniadocuments/(Public Health in the 21st Century) Micaela L. Surez_ Steffani M. Ortega - Pneumonia_ Symptoms Diagnosis and Treatment _ Symptoms Diagnosis and Treatment-Nova Science Publishers Incorporated (2011.pdf',
 'page': 0}

In [7]:
data = [{
    'chunk': chunk.page_content,
    'source': chunk.metadata.get('source')
} for chunk in chunks]

In [8]:
import pandas as pd
docs = pd.DataFrame(data)
docs.head()

,chunk,source
0,"Pneumonia: Symptoms, Diagnosis and Treatment :...",/kaggle/input/pneumoniadocuments/(Public Healt...
1,"Pneumonia: Symptoms, Diagnosis and Treatment :...",/kaggle/input/pneumoniadocuments/(Public Healt...
2,PUBLIC HEALTH IN THE 21ST CENTURY \n \n \n \n...,/kaggle/input/pneumoniadocuments/(Public Healt...
3,PUBLIC HEALTH IN THE 21ST CENTURY \n \nAdditi...,/kaggle/input/pneumoniadocuments/(Public Healt...
4,PUBLIC HEALTH IN THE 21ST CENTURY \n \n \n \n...,/kaggle/input/pneumoniadocuments/(Public Healt...


In [9]:
from langchain_community.document_loaders import DataFrameLoader

loader = DataFrameLoader(docs, page_content_column="chunk")
documents = loader.load()

In [10]:
documents[0]

Document(metadata={'source': '/kaggle/input/pneumoniadocuments/(Public Health in the 21st Century) Micaela L. Surez_ Steffani M. Ortega - Pneumonia_ Symptoms Diagnosis and Treatment _ Symptoms Diagnosis and Treatment-Nova Science Publishers Incorporated (2011.pdf'}, page_content='Pneumonia: Symptoms, Diagnosis and Treatment : Symptoms, Diagnosis and Treatment, edited by Micaela L. Suárez, and Steffani M. Ortega, Nova\nCopyright © 2011. Nova Science Publishers, Incorporated. All rights reserved.')

In [11]:
documents[0].metadata

{'source': '/kaggle/input/pneumoniadocuments/(Public Health in the 21st Century) Micaela L. Surez_ Steffani M. Ortega - Pneumonia_ Symptoms Diagnosis and Treatment _ Symptoms Diagnosis and Treatment-Nova Science Publishers Incorporated (2011.pdf'}

In [12]:
print(len(docs))
print(len(docs['chunk'][6]))

171
1274


In [13]:
# NVIDIA NIM API
from openai import OpenAI
import json

with open('config.json', 'r') as file:
    config = json.load(file)

# url = config['qdrant_url']
# api_key = config['qdrant_api_key']


client = OpenAI(
  base_url = config['nvidia_base_url'],
  api_key = config['nvidia_api_key']
)

In [228]:
# documents = documents[16:17]

In [ ]:
embeddings = client.embeddings.create(
    input=[doc.page_content for doc in documents],
    model="nvidia/nv-embedqa-e5-v5",
    encoding_format="float",
    extra_body={"input_type": "query", "truncate": "NONE"}
)

embeddings_data = embeddings.data
embedding_vectors = [e.embedding for e in embeddings_data]

In [197]:
len(embedding_vectors[0]) if embedding_vectors else 0

1024

In [210]:
from qdrant_client.models import PointStruct
import uuid

# Create PointStruct objects
points = [
    PointStruct(
        id=str(uuid.uuid4()),  # Use a unique ID for each document
        vector=embedding,  # Ensure this is a list of floats
        payload={"text": doc.page_content}  # Optional metadata
    )
    for i, (doc, embedding) in enumerate(zip(documents, embedding_vectors))
]

In [211]:
from qdrant_client import QdrantClient
import json

with open('config.json', 'r') as file:
    config = json.load(file)

url = config['qdrant_url']
api_key = config['qdrant_api_key']

# Initialize Qdrant client
qdrant_client = QdrantClient(url=url, api_key=api_key)

In [87]:
from qdrant_client.models import CollectionParams, VectorParams

# Define the dimension of your vectors and distance metric
vector_dim = 1024  # Dimension of your vectors
distance_metric = "Cosine"  # Distance metric (use "Cosine" as string)

# Define vector configuration
vector_params = VectorParams(size=vector_dim, distance=distance_metric)

# Create the collection
qdrant_client.create_collection(
    collection_name="pneuBot",
    vectors_config=vector_params  # Correct argument name
)

True

In [212]:
# Insert points into Qdrant
qdrant_client.upsert(
    collection_name="pneuBot",
    points=points
)

UpdateResult(operation_id=8, status=<UpdateStatus.COMPLETED: 'completed'>)

In [217]:
query = "How pneumonia will be treated?"
# client.similarity_search(query, k=3)
# Embed the query
query_embedding_response = client.embeddings.create(
    input=[query],  # Embed the query text
    model="nvidia/nv-embedqa-e5-v5",
    encoding_format="float",
    extra_body={"input_type": "query", "truncate": "NONE"}
)

# Extract the embedding vector from the response
query_embedding = query_embedding_response.data[0].embedding

# Perform similarity search
results = qdrant_client.search(
    collection_name="pneuBot",
    query_vector=query_embedding,
#     k=3
)
# # Print results
# for result in results:
#     print(result)

In [218]:
def custom_prompt(query: str):
    query_embedding_response = client.embeddings.create(
        input=[query],
        model="nvidia/nv-embedqa-e5-v5",
        encoding_format="float",
        extra_body={"input_type": "query", "truncate": "NONE"}
    )
    
    # Extract the embedding vector from the response
    query_embedding = query_embedding_response.data[0].embedding
    
    # Perform similarity search
    results = qdrant_client.search(
        collection_name="pneuBot",
        query_vector=query_embedding,  # Ensure you pass the query embedding here
#         top_k=3
    )
    
    # Extract the page content from the results
    source_knowledge = "\n".join([x.payload['text'] for x in results])
    
    # Create the augmented prompt
    augment_prompt = f"""Using the contexts below, answer the query:

    Contexts:
    {source_knowledge}

    Query: {query}"""
    
    return augment_prompt

In [204]:
# print(custom_prompt(query))

In [219]:
messages = []

In [220]:
# messages.append({"role":"assistant", "content": full_response})
prompt = {"role":"system", "content": custom_prompt(query)}
messages.append(prompt)

res = client.chat.completions.create(
  model="meta/llama-3.1-8b-instruct",
  messages=messages,
  temperature=0.2,
  top_p=0.7,
  max_tokens=1024,
  stream=True
)

full_response = ""
for chunk in res:
  if chunk.choices[0].delta.content is not None:
#     print(chunk.choices[0].delta.content, end="")
    full_response += chunk.choices[0].delta.content
print(full_response)

According to the provided text, the treatment of pneumonia will depend on the etiological diagnosis. The authors mention that various diagnostic tests, both invasive and non-invasive, are available to guide the clinical diagnosis of pneumonia and establish the optimal treatment. However, they do not provide specific information on the treatment of pneumonia.

To answer the query, we can infer that the treatment of pneumonia will involve a combination of supportive treatment and specific treatment based on the underlying cause of the infection. The authors mention that supportive treatment is discussed in Chapter II, titled "Supportive Treatment to Improve Outcome of Pneumonia" by Michael Eisenhut and Tomasz Rajkowski.

Therefore, the treatment of pneumonia will likely involve a combination of:

1. Supportive treatment, which is discussed in Chapter II.
2. Specific treatment based on the etiological diagnosis, which may involve antibiotics, antiviral medications, or other treatments dep